# Test Dataset Preprocessing

The test data preprocessing is the same as training data preprocessing, however it should performed after the training data
preprocessing pipeline has finished, in order to avoid data leakage and overfitting!

# Defining Helper Functions

In [10]:
from py_stringmatching.similarity_measure.monge_elkan import MongeElkan
import pandas as pd
import numpy as np
import re
import warnings

warnings.filterwarnings('ignore')


def find_k_nearest_by_coordinates(
        samples_df: pd.DataFrame,
        k_nearest: int,
        longitude: float,
        latitude: float
) -> pd.DataFrame:
    geo_coordinates = samples_df[['longitude', 'latitude']]
    target_coord = np.float32([longitude, latitude])
    distances = np.sqrt(np.sum(np.power(target_coord - geo_coordinates, 2), axis=1))
    min_distance_indices = np.argpartition(distances, k_nearest)[1: k_nearest+1]
    return samples_df.iloc[min_distance_indices]


def impute_zeros_by_nearby_samples(
        samples_df: pd.DataFrame,
        row: pd.Series,
        location_column: str,
        target_column: str,
        std_threshold: float or None
) -> (pd.Series, bool):
    imputed = False
    area = row[location_column]
    target_values = samples_df.loc[samples_df[location_column] == area, target_column]

    if target_values.shape[0] > 1:
        non_zero_ids = target_values > 0

        if non_zero_ids.sum() > 0:
            non_zero_values = target_values[non_zero_ids]

            if std_threshold is not None and np.std(non_zero_values) > std_threshold:
                return row, False
            row[target_column] = non_zero_values.mean()
            imputed = True
    return row, imputed


def compute_recorded_season_and_operation_time(samples_df: pd.DataFrame) -> (pd.DataFrame, int):
    seasons = {
        1: 0, 2: 0,
        3: 1, 4: 1, 5: 1,
        6: 2, 7: 2, 8: 2, 9: 2, 10: 2,
        11: 3, 12: 3,
    }

    samples_df['seasons'] = pd.DataFrame({'Months': pd.DatetimeIndex(samples_df['date_recorded']).month})
    samples_df['seasons'] = samples_df['seasons'].apply(lambda month: seasons[month])

    samples_df['construction_year'] = samples_df['construction_year'].replace({0: 10000})
    samples_df['operation_time'] = pd.DatetimeIndex(samples_df['date_recorded']).year - samples_df['construction_year']
    samples_df.loc[samples_df['operation_time'] < 0, 'operation_time'] = -1
    return samples_df, (samples_df['operation_time'] == -1).sum()


def preprocess_single_token(token: str) -> str:
    non_allowed = ['.', '/', '\'', ',', '&', '(', ')', '-', '_', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    for ch in non_allowed:
        token = token.replace(ch, ' ')
    token = re.sub('\\s+', ' ', token).lower()
    return token.lower()


def cluster_similar_tokens(
        samples_df: pd.DataFrame,
        preprocessed_token_with_counts: pd.Series,
        target_column: str,
        similarity_threshold: int
) -> (pd.DataFrame, dict, int):
    clusters = {}

    similarity_algorithm = MongeElkan()
    num_clustered = 0

    original_token_counts = samples_df[target_column].value_counts()
    preprocess_token_list = list(preprocessed_token_with_counts.index)

    for original_token, original_token_count in original_token_counts.items():
        preprocessed_token = preprocess_single_token(token=original_token)

        similarities = {
            token2: similarity_algorithm.get_raw_score(preprocessed_token.split(' '), token2.split(' '))
            for token2 in preprocess_token_list if preprocessed_token != token2
        }

        most_similar_token = max(similarities, key=similarities.get)
        similarity_score = similarities[most_similar_token]

        if similarity_score >= similarity_threshold and original_token_count < preprocessed_token_with_counts[most_similar_token]:
            clusters[original_token] = most_similar_token
            num_clustered += 1
        else:
            clusters[original_token] = preprocessed_token

    samples_df[target_column] = samples_df[target_column].apply(lambda token: clusters[token])
    return samples_df, num_clustered


def impute_public_meeting_and_permit(samples_df: pd.DataFrame) -> (pd.DataFrame, int, int):
    for target_column in ['public_meeting', 'permit']:
        samples_df[target_column] = samples_df[target_column].fillna('Unknown')
        samples_df[target_column] = samples_df[target_column].replace({'False': 0, 'True': 1, 'Unknown': -1})
    return samples_df, (samples_df['public_meeting'] == -1).sum(), (samples_df['permit'] == -1).sum()


def impute_nan_row_by_nearby_samples(
        samples_df: pd.DataFrame,
        row: pd.Series,
        location_column: str,
        target_column: str,
        frequency_threshold: float
) -> (pd.Series, bool):
    imputed = False

    area = row[location_column]
    target_values = samples_df.loc[samples_df[location_column] == area, target_column]

    if target_values.shape[0] > 1:
        not_na_ids = target_values.notna()

        if not_na_ids.sum() > 0:
            value_frequencies = target_values.value_counts() / target_values.shape[0]
            non_nan_values = target_values[not_na_ids]
            most_frequent_value = non_nan_values.mode()

            if frequency_threshold is not None and value_frequencies[most_frequent_value].tolist()[0] < frequency_threshold:
                return row, False
            row[target_column] = most_frequent_value
            imputed = True
    return row, imputed


def impute_unknown_row_by_nearby_samples(
        samples_df: pd.DataFrame,
        row: pd.Series,
        location_column: str,
        target_column: str,
        frequency_threshold: float
) -> (pd.Series, bool):
    imputed = False

    area = row[location_column]
    target_values = samples_df.loc[samples_df[location_column] == area, target_column]

    if target_values.shape[0] > 1:
        not_unknown_ids = target_values != 'unknown'

        if not_unknown_ids.sum() > 0:
            value_frequencies = target_values.value_counts() / target_values.shape[0]
            most_frequent_value_frequency = value_frequencies.values[0]

            if value_frequencies.index[0] != 'unknown' and most_frequent_value_frequency >= frequency_threshold:
                row[target_column] = value_frequencies.index[0]
                imputed = True
    return row, imputed

# Importing Train-Test Datasets

In [11]:
import pandas as pd

train_inputs = pd.read_csv('train_inputs.csv')
test_samples = pd.read_csv('test.csv')

train_inputs.shape, test_samples.shape

((59400, 42), (14850, 40))

# Imputing Amount TSH, Population, GPS Height

In [12]:
amount_tsh_std_threshold = 50
population_std_threshold = 50

for target_column, std_threshold in zip(['amount_tsh', 'population'], [amount_tsh_std_threshold, population_std_threshold]):
    num_imputed = 0

    zero_ids = np.where(test_samples[target_column] == 0)[0]
    for zero_id in zero_ids:
        row = test_samples.iloc[zero_id]

        row, imputed = impute_zeros_by_nearby_samples(
            samples_df=train_inputs,
            row=row,
            location_column='subvillage',
            target_column=target_column,
            std_threshold=std_threshold
        )
        test_samples.iloc[zero_id] = row
        num_imputed += imputed
    print(f'Imputed {num_imputed} values of "{target_column}" using nearby "subvillage" samples')

for location_column in ['subvillage', 'ward', 'lga', 'district_code']:
    num_imputed = 0

    zero_ids = np.where(test_samples['gps_height'] == 0)[0]
    for zero_id in zero_ids:
        row = test_samples.iloc[zero_id]

        row, imputed = impute_zeros_by_nearby_samples(
            samples_df=train_inputs,
            row=row,
            location_column=location_column,
            target_column='gps_height',
            std_threshold=None
        )
        test_samples.iloc[zero_id] = row
        num_imputed += imputed
    print(f'Imputed {num_imputed} values of "gps_height" using nearby "{location_column}" samples')

Imputed 1039 values of "amount_tsh" using nearby "subvillage" samples
Imputed 306 values of "population" using nearby "subvillage" samples
Imputed 3337 values of "gps_height" using nearby "subvillage" samples
Imputed 1866 values of "gps_height" using nearby "ward" samples
Imputed 8 values of "gps_height" using nearby "lga" samples
Imputed 0 values of "gps_height" using nearby "district_code" samples


# Computing Recorded Season, Operation Time

In [13]:
test_samples, num_invalid_operation_times = compute_recorded_season_and_operation_time(samples_df=test_samples)
f'{num_invalid_operation_times.sum()} invalid dates were set to -1'

'5263 invalid dates were set to -1'

# # Funder & Installer

In [14]:
funder_similarity_threshold = 0.5
installer_similarity_threshold = 0.5

for token_column, similarity_threshold in zip(['funder', 'installer'], [funder_similarity_threshold, installer_similarity_threshold]):
    print(f'Number of unique {token_column} = {test_samples[token_column].unique().shape[0]} before clustering')

    test_samples[token_column] = test_samples[token_column].fillna('unknown')
    test_samples[token_column] = test_samples[token_column].replace({'0': 'other', '-1': 'other'})

    num_clustered = 1000
    num_iteration_clustered = 1
    iteration = 0

    while num_clustered > 0 and num_clustered != num_iteration_clustered:
        num_clustered = num_iteration_clustered

        test_samples, num_iteration_clustered = cluster_similar_tokens(
            samples_df=test_samples,
            preprocessed_token_with_counts=train_inputs[token_column].value_counts(),
            target_column=token_column,
            similarity_threshold=similarity_threshold
        )

        iteration += 1
        print(f'Iteration = {iteration}, Clustered {num_iteration_clustered} tokens')

    for token, count in test_samples[token_column].value_counts().items():
        if count < 10:
            test_samples.loc[test_samples[token_column] == token, token_column] = 'other'

    print(f'Number of unique {token_column} = {test_samples[token_column].unique().shape[0]} after clustering')

Number of unique funder = 981 before clustering
Iteration = 1, Clustered 933 tokens
Iteration = 2, Clustered 136 tokens
Iteration = 3, Clustered 47 tokens
Iteration = 4, Clustered 23 tokens
Iteration = 5, Clustered 15 tokens
Iteration = 6, Clustered 12 tokens
Iteration = 7, Clustered 11 tokens
Iteration = 8, Clustered 11 tokens
Number of unique funder = 77 after clustering
Number of unique installer = 1092 before clustering
Iteration = 1, Clustered 1057 tokens
Iteration = 2, Clustered 148 tokens
Iteration = 3, Clustered 50 tokens
Iteration = 4, Clustered 18 tokens
Iteration = 5, Clustered 11 tokens
Iteration = 6, Clustered 10 tokens
Iteration = 7, Clustered 10 tokens
Number of unique installer = 65 after clustering


# Imputing Public Meeting & Permit

In [15]:
test_samples, num_public_meeting_imputed, num_permit_imputed = impute_public_meeting_and_permit(samples_df=test_samples)
print(f'Number of Public Meeting Imputed: {num_public_meeting_imputed}')
print(f'Number of Permit Imputed: {num_permit_imputed}')

Number of Public Meeting Imputed: 821
Number of Permit Imputed: 737


# Scheme Management

In [16]:
scheme_management_frequency_threshold = 0.7

for location_column in ['subvillage', 'ward']:
    nan_ids = np.where(test_samples['scheme_management'].isna())[0]

    for nan_ids in nan_ids:
        row = test_samples.iloc[nan_ids]

        row, imputed = impute_nan_row_by_nearby_samples(
            samples_df=train_inputs,
            row=row,
            location_column=location_column,
            target_column='scheme_management',
            frequency_threshold=scheme_management_frequency_threshold
        )
        test_samples.iloc[nan_ids] = row
        num_imputed += imputed
    print(f'Imputed {num_imputed} values of "scheme_management" using nearby "{location_column}" samples')

num_nans = test_samples['scheme_management'].isna().sum()
test_samples['scheme_management'] = test_samples['scheme_management'].fillna('unknown')
print(f'NaNs converted to "Unknown" = {num_nans}')

Imputed 364 values of "scheme_management" using nearby "subvillage" samples
Imputed 789 values of "scheme_management" using nearby "ward" samples
NaNs converted to "Unknown" = 180


# Management, Payment Type, Water Quality, Quantity

In [17]:
std_threshold = 0.8

for location_col in ['subvillage', 'ward']:
    for target_col in ['management', 'payment_type', 'water_quality', 'quantity']:
        unknown_ids = np.where(test_samples['scheme_management'].isna())[0]

        for unknown_ids in unknown_ids:
            row = test_samples.iloc[unknown_ids]

            row, imputed = impute_nan_row_by_nearby_samples(
                samples_df=train_inputs,
                row=row,
                location_column=location_col,
                target_column=target_col,
                frequency_threshold=scheme_management_frequency_threshold
            )
            test_samples.iloc[unknown_ids] = row
            num_imputed += imputed
        print(f'Imputed {num_imputed} values of "{target_col}" using nearby "{location_column}" samples')

Imputed 789 values of "management" using nearby "ward" samples
Imputed 789 values of "payment_type" using nearby "ward" samples
Imputed 789 values of "water_quality" using nearby "ward" samples
Imputed 789 values of "quantity" using nearby "ward" samples
Imputed 789 values of "management" using nearby "ward" samples
Imputed 789 values of "payment_type" using nearby "ward" samples
Imputed 789 values of "water_quality" using nearby "ward" samples
Imputed 789 values of "quantity" using nearby "ward" samples


# Storing Dataset

In [18]:
test_samples.to_csv('test_inputs.csv', index=False)
test_samples

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,seasons,operation_time
0,50785,5500.0,2013-02-04,w d i,1996.0,tasaf dmdd,35.290799,-4.059696,Dinamu Secondary School,0,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other,0,1
1,51630,0.0,2013-02-04,government of tanzania,1569.0,dwe,36.656709,-3.309214,Kimnyak,0,...,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,0,13
2,17168,0.0,2013-02-01,unknown,1567.0,unknown,34.767863,-5.004344,Puma Secondary,0,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other,0,3
3,45559,0.0,2013-01-22,ministry of water,267.0,fini water,38.058046,-9.418672,Kwa Mzee Pange,0,...,good,dry,dry,shallow well,shallow well,groundwater,other,other,0,26
4,49871,500.0,2013-03-27,redep,1260.0,w c s,35.006123,-10.950412,Kwa Mzee Turuka,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,50.0,2011-02-24,danida,34.0,da,38.852669,-6.582841,Kwambwezi,0,...,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,0,23
14846,18990,1000.0,2011-03-21,the people of japan,1282.0,halmashauri ya wilaya sikonge,37.451633,-5.350428,Bonde La Mkondoa,0,...,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,1,17
14847,28749,0.0,2013-03-04,unknown,1476.0,unknown,34.739804,-4.585587,Bwawani,0,...,good,insufficient,insufficient,dam,dam,surface,communal standpipe,communal standpipe,1,3
14848,33492,2000.0,2013-02-18,oikos e afrika,998.0,dwe,35.432732,-10.584159,Kwa John,0,...,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,0,4
